In [1]:
import numpy as np   # Importo paquete de álgebra lineal
import matplotlib.pyplot as plt   # Importo paquete de gráficos
import copy
import mef
plt.rc('figure', figsize=(15,8))   # Para gráficos
plt.rc('font',size=22)             # Para gráficos

## Punto 2

Considere el entramado mostrado en la figura, con una fuerza aplicada de $20\:kN$. Calcule los desplazamientos de cada uno de los nodos y las tensiones que sufre cada elemento. Todos los elementos tienen $E = 210\:GPa$ y una sección de $10\:cm^2$, excepto el elemento 3, que tiene una sección de $20\:cm^2$. Los elementos 2 y 5 tienen una longitud de $8\:m$ y el elemento 3 de $4\:m$.
<center><img src="ImgProb02.png"></center>

In [2]:
# Defino la matriz de nodos "MN".
# - Columna 1 es la coordenada "x" del nodo.
# - Columna 2 es la coordenada "y" del nodo.
# - Columna 3 es la coordenada "z" del nodo.
MN = np.array([[0,0,0],
               [8,0,0],
               [16,0,0],
               [8,4,0]])

# Su número de filas es el número de NODOS "Nn".
Nn = MN.shape[0]

In [3]:
# Defino la matriz de conectividad "MC".
# Informa qué nodos componen a cada elemento.
MC= np.array([[0,3],
              [0,1],
              [1,3],
              [2,3],
              [1,2]])

# Su número de filas es el número de ELEMENTOS "Ne".
# Su número de columnas es el número NODOS POR ELEMENTO "Nnxe".
Ne, Nnxe = MC.shape

In [4]:
# Defino los grados de libertad por nodo "glxn".
glxn = 2

In [5]:
# Defino el módulo de elasticidad "E" de cada elemento.
E = np.array([210,210,210,210,210])*1e9   # Pa

# Defino la sección "A" de cada elemento.
A = np.array([10,10,20,10,10])*1e-4   # m^2

# Defino la longitud "L" de cada elemento.
L = np.array([8,8,4,8,8])*1e9   # m

### Condiciones de vínculo en DESPLAZAMIENTO
- $x_0 = 0\: m$
- $y_0 = 0\: m$
- $y_2 = 0\: m$

### Condiciones de vínculo en FUERZA
- $F_{1x} = 0\: kN$
- $F_{1y} = -20\: kN$
- $F_{2x} = 0\: kN$
- $F_{3x} = 0\: kN$
- $F_{3y} = 0\: kN$

In [6]:
# Defino vector "s" que contiene los nodos con condiciones de vínculo en desplazamiento.
s = np.array([0,1,5])

# Defino vector "Us" con los valores de las condiciones de vínculo.
Us = [[0],[0],[0]]

# Defino vector "r" que contiene los nodos con condiciones de vínculo en fuerza. Es el complemento de "s".
r = np.array([i for i in range(Nn*glxn) if i not in s])

# Defino vector "Fr" con los valores de las condiciones de vínculo.
Fr = [[0],[-20000],[0],[0],[0]]

## Resolución

In [7]:
# Llamo al paquete "mef", que contiene la función "Kglobal" que calcula la matriz global "K".
K = mef.Kglobal(MN, MC, E, A, glxn)

In [8]:
# Llamo al paquete "mef", que contiene la función "solve" que calcula los vectores de fuerzas "F" y de desplazamientos 
# "U", empleando MEF. 
F, U = mef.solve(K, r, Fr, s, Us)

In [9]:
print('Las FUERZAS (en N) son:')
print(F)
print('Los DESPLAZAMIENTOS (en m) son:')
print(U)

Las FUERZAS (en N) son:
[[ 7.27595761e-12]
 [ 1.00000000e+04]
 [ 0.00000000e+00]
 [-2.00000000e+04]
 [ 0.00000000e+00]
 [ 1.00000000e+04]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
Los DESPLAZAMIENTOS (en m) son:
[[ 0.        ]
 [ 0.        ]
 [ 0.0007619 ]
 [-0.00384387]
 [ 0.00152381]
 [ 0.        ]
 [ 0.0007619 ]
 [-0.0036534 ]]


In [10]:
# FORMA MÁS LINDA DE PRESENTAR LOS DATOS:
# - "%s" significa que te pone número entero.
# - "%.4f" significa que te pone número con 4 cifras decimales.
# - "%2.4f" y "%7.4f" sólo varían en que con "7" te pone los números alineados respecto del "=" y queda más lindo si 
# llega  a haber uno con signo "-".
for nodo in range(Nn):
    print('Nodo %s     Ux = %2.4f mm     Uy = %7.4f mm     Fx = %2.4f N     Fy = %7.4f N'%(nodo+1, U[2*nodo]*1000, U[2*nodo+1]*1000, F[2*nodo], F[2*nodo+1]))

Nodo 1     Ux = 0.0000 mm     Uy =  0.0000 mm     Fx = 0.0000 N     Fy = 10000.0000 N
Nodo 2     Ux = 0.7619 mm     Uy = -3.8439 mm     Fx = 0.0000 N     Fy = -20000.0000 N
Nodo 3     Ux = 1.5238 mm     Uy =  0.0000 mm     Fx = 0.0000 N     Fy = 10000.0000 N
Nodo 4     Ux = 0.7619 mm     Uy = -3.6534 mm     Fx = 0.0000 N     Fy =  0.0000 N


### Problema

Me di cuenta que tenía hecho el ejercicio hasta la línea anterior, pero hoy leí mejor el enunciado y **pedía tensiones de cada elemento también**.

Intenté calcularlas tal y como lo habíamos hecho en el ejercicio 3, es decir, en multiplicando el módulo de elasticidad por la deformación del elemento, calculada como el cociente entre la diferencia de desplazamientos y la longitud del elemento:

Luego me di cuenta que acá no tenemos todos los elementos alineados en el mismo eje, así que no iba a servir. Mi duda es si debo obtener:
- Una única tensión por elemento, que vendría de calcular la deformación en el eje del elemento (proyectando manualmente los desplazamientos de los nodos correspondientes a ese elemento; esta parte me genera dudas si es correcta) y multiplicando por el módulo de elasticidad.
- Dos tensiones por elemento, es decir, una tensión en "X" y otra en "Y" (me da más dudas esta opción pero se me ocurre), para lo cual ya poseería los desplazamientos en "X" y en "Y" (calculados con MEF) pero me faltaría conocer la longitud del elemento proyectada en "X" y en "Y".

Es decir, ambas opciones serían difíciles de generalizar, debería calcular medio a mano las tensiones, todo por el hecho de que no tengo todos los elementos alineados a lo largo de un mismo eje (tengo dos elementos puramente en "X", un elemento puramente en "Y" y dos elementos en ambos ejes). ¿Voy bien encaminado con la forma de encarar el cálculo de las tensiones? ¿O me estoy rebuscando mucho con las dos opciones que planteé? Tal vez hay alguna manera más fácil de hacerlo.

<div style="background:#999900"> una tension x y una tensión y no tiene mucho sentido. Sin embargo, podes calcular el $\Delta L$ del elemento tomando las coordenadas luego de la deformación </div>

<div style="background:#999900">
$$
MN' = MN + U^{2D}
$$
</div>

<div style="background:#999900">
donde $U^{2D}$ son las deformaciones reexpresadas en 2D,
</div>

<div style="background:#999900">
    $U^{2D} = U.reshape(NNODOS, 3)$
</div>

<div style="background:#999900">
de manera que la nueva matriz de nodos $MN'$  son las nuevas posiciones de los nodos. Luego, podes calcular las luevas longitudes de los elementos, py por último  las $\Delta L$
 </div>